# 신경망의 구현 과정
***1*** | **환경 변수 지정**:  데이터셋(입력 값, 결괏값),
     학습률, 활성화 함수, 가중치 포함.
<br>
***2*** | **신경망 실행**: 초기값 입력 -> 활성화 함수, 가중치 -> 결과값
<br>
***3*** | **결과를 실제 값과 비교**: 오차 측정
<br>
***4*** | **역전파 실행**: 출력층, 은닉층의 가중치 수정
<br>
***5*** | **결과 출력**

# 0. 라이브러리 import

In [48]:
import random
import numpy as np

random.seed(777) # 시드: 난수 생성기를 초기화해주는 역할

#num_x=2+1
#num_yh=2
#num_yo=1

#input=[1.0]*num_x
#hidden=[1.0]*num_yh
#out=[1.0]*num_yo

#fill=0.0
#weight_in = []
#for i in range(num_x):
#  weight_in.append([fill]*num_yh)

#for i in range(num_x):
#        for j in range(num_yh):
#          weight_in[i][j] = random.random()

#weight_in

[[0.22933408950153078, 0.44559617334521107],
 [0.36859824937216046, 0.269835098321503],
 [0.3361436466700177, 0.7523163560031157]]

# 1. 환경 변수 설정하기

### XOR 진리표
| x1 | X2 | 결괏값 |
| --- | --- | --- | 
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

In [30]:
# 입력 값, 타깃 값
# 입력 값은 진리표의 x1, x2
# 타깃 값은 정답 데이터(yt)

data = [
        [[0,0],[0]],
        [[0,1],[1]],
        [[1,0],[1]],
        [[1,1],[0]]
]

# 실행 횟수(iterations), 학습률(lr), 모멘텀 계수(mo) 설정
iterations = 5000
lr = 0.1
mo = 0.4

# 활성화 함수, 초기 가중치 지정
# 활성화 함수 1. 시그모이드
def sigmoid(x, derivative = False):
  if (derivative == True):
    return x*(1-x)  # 미분할 때의 값
  return 1/(1+np.exp(-x)) # 미분하지 않을 때의 값

# 활성화 함수 2. tanh(하이퍼볼릭 탄젠트)
def tanh(x, derivative=False):
  if (derivative == True):
    return 1-x**2 # 미분할 때의 값: 1-(출력의 제곱)
  return np.tanh(x)

# 가중치 배열을 만드는 함수
def makeMatrix(i,j,fill=0.0): # i개 행 j개 열(i×j)
  mat = []
  for i in range(i):
    mat.append([fill]*j)
  return  mat

# 2. 신경망의 실행
신경망을 실행하는 클래스는
- 초깃값의 지정(입력값,은닉층의 초깃값, 출력층의 초깃값, 바이어스, 활성화 함수와 가중치 초깃값)
- 업데이트 함수(optimizer 선택 가능 -> 모멘텀 SGD)
- 역전파 함수
로 구성된다.

In [37]:
# 신경망의 실행
class NeuralNetwork:

    # 초깃값 지정
    def __init__(self, num_x, num_yh, num_yo, bias=1):

      # 입력 값(num_x), 은닉층의 초깃값(num_yh), 출력층의 초깃값(num_yo), 바이어스
      self.num_x = num_x + bias # 바이어스는 1로 설정
      ### Q. 왜 여기에 bias를 더함?
      self.num_yh = num_yh
      self.num_yo = num_yo

      # 활성화 함수 초깃값
      self.activation_input = [1.0]*self.num_x
      self.activation_hidden = [1.0]*self.num_yh
      self.activation_out = [1.0]*self.num_yo

      # 가중치 입력 초깃값
      self.weight_in = makeMatrix(self.num_x, self.num_yh)  # 입력값 개수만큼의 행, 레이어 개수만큼의 열 -> 행열
      for i in range(self.num_x):
        for j in range(self.num_yh):
          self.weight_in[i][j] = random.random()

      # 가중치 출력 초깃값
      self.weight_out = makeMatrix(self.num_yh,self.num_yo) # 레이어 개수만큼의 행, 출력값 개수만큼의 열 -> 행열
      for j in range(self.num_yh):
        for k in range(self.num_yo):
          self.weight_out[j][k] = random.random()

      # 모멘텀 SGD를 위한 이전 가중치 초깃값
      self.gradient_in = makeMatrix(self.num_x, self.num_yh)
      self.gradient_out = makeMatrix(self.num_yh, self.num_yo)



    # 업데이트 함수
    def update(self, inputs):

      # 입력층의 활성화 함수
      for i in range(self.num_x-1):
        self.activation_input[i]=inputs[i]

      # 은닉층의 활성화 함수
      for j in range(self.num_yh):
        sum=0.0
        for i in range(self.num_x):
          sum+=self.activation_input[i]*self.weight_in[i][j]
        # 시그모이드와 tanh 중에서 활성화 함수 선택
        self.activation_hidden[j]=tanh(sum, False) 

      # 출력층의 활성화 함수
      for k in range(self.num_yo):
        sum = 0.0
        for j in range(self.num_yh):
          sum += self.activation_hidden[j]*self.weight_out[j][k]
        # 시그모이드와 tanh 중에서 활성화 함수 선택
        self.activation_out[k]=tanh(sum, False) 

      return self.activation_out[:]


    # 역전파 실행
  
    def backPropagate(self, targets):
    
      # 델타 출력 계산
      output_deltas = [0.0]*self.num_yo
      for k in range(self.num_yo):
        error = targets[k] - self.activation_out[k]
        # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
        output_deltas[k] = tanh(self.activation_out[k],True)*error
      
      # 은닉 노드의 오차 함수
      hidden_deltas = [0.0]*self.num_yh
      for j in range(self.num_yh):
        error=0.0
        for k in range(self.num_yo):
          error += output_deltas[k]*self.weight_out[j][k]
          # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
          hidden_deltas[j] = tanh(self.activation_hidden[j],True)*error

      # 출력 가중치 업데이트
      for j in range(self.num_yh):
        for k in range(self.num_yo):
          gradient = output_deltas[k]*self.activation_hidden[j]
          v = mo*self.gradient_out[j][k] - lr*gradient
          self.weight_out[j][k] += v
          self.gradient_out[j][k] = gradient

      # 입력 가중치 업데이트
      for i in range(self.num_x):
        for j in range(self.num_yh):
          gradient = hidden_deltas[j]*self.activation_input[i]
          v = mo*self.gradient_in[i][j]-lr*gradient
          self.weight_in[i][j]+= v
          self.gradient_in[i][j]= gradient

      # 오차 계산(최소 제곱법)
      error = 0.0
      for k in range(len(targets)):
        error +=0.5*(targets[k]-self.activation_out[k])**2
      return error

  # 학습 실행
    def train(self, patterns):
      for i in range(iterations):
        error = 0.0
        for p in patterns:
          inputs = p[0]
          targets = p[1]
          self.update(inputs)
          error += self.backPropagate(targets)
        if i%500 == 0:
          print('error: %-.5f' %error)

  # 결괏값 출력
    def result(self, patterns):
      for p in patterns:
        print('Input: %s, Predict: %s -> %s' %(p[0],self.update(p[0]),np.round(self.update(p[0]),1)))


# 3. 학습 실행시키고 결괏값 출력

In [38]:

if __name__ == '__main__':
  # 두 개의 입력값, 두 개의 레이어, 하나의 출력값 갖도록 설정
  n = NeuralNetwork(2,2,1)

  # 학습 실행
  n.train(data)

  # 결괏값 출력
  n.result(data)

error: 0.48996
error: 0.00269
error: 0.00089
error: 0.00051
error: 0.00036
error: 0.00027
error: 0.00022
error: 0.00019
error: 0.00016
error: 0.00014
Input: [0, 0], Predict: [0.0005987966863271798] -> [0.]
Input: [0, 1], Predict: [0.9889458320792305] -> [1.]
Input: [1, 0], Predict: [0.9889765087616103] -> [1.]
Input: [1, 1], Predict: [0.0021738653536299353] -> [0.]
